<a href="https://colab.research.google.com/github/Alexmarco-gif/pytorch-deep-learning-professional-path/blob/main/Advanced_Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Module 1: Model Deployment & ONNX
Saving and Exporting the Final Model <br>
Task is to practice the serialization and export process using the ResNet-18 model trained in the previous Capstone project. You may not have the weights loaded right now, we will simulate the process by loading a fresh ResNet-18 instannce.

Instruction:
1. Instantiate the pre-trained ResNet-18 model and modify its final fc layer to output 10 classes.
2. Set the model to evaluation mode (.eval()).
3. Save the model's state dictionary to a file named resnet18_final.pth.
4. Define a dummy input tensor matching the required input shape for CIFAR-10 images (Batch size 1, 3 channels, 32x32).
5. Export the model to the ONNX format, naming the file resnet18_cifar10.onnx.

In [ ]:
!pip install onnx
import torch
import torchvision.models as models

# 1. Instantiate the pre-trained ResNet-18 model
model = models.resnet18(pretrained=True)

# Modify its final fc layer to output 10 classes (for CIFAR-10)
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 10)
model.eval()


torch.save(model.state_dict(), 'resnet18_cifar10.pth')

dummy_input = torch.rand(1, 3, 32, 32)

torch.onnx.export(
    model,
    dummy_input,
    'resnet18_cifar10.onnx',
    export_params =True,
    opset_version=17,      # ONNX standard version
    do_constant_folding=True, # Optimize graph
    input_names=['input'],
    output_names=['output']
    )


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/tmp/ipython-input-1299438506.py:18: DeprecationWarning: You are using the legacy TorchScript-based ONNX export. Starting in PyTorch 2.9, the new torch.export-based ONNX exporter will be the default. To switch now, set dynamo=True in torch.onnx.export. This new exporter supports features like exporting LLMs with DynamicCache. We encourage you to try it and share feedback to 

## Module 2: Recurrent Neural Networks (RNNs)
Recurrent Neural Networks (RNNs) are designed specifically to handle sequences by maintaining an internal hidden state that acts as a memory of past inputs.
1. The Simple RNN
A simple RNN processes data one step (token/word/timestamp) at a time. At each step $t$, it takes the current input $x_t$ and the hidden state from the previous step $h_{t-1}$ to calculate the new hidden state $h_t$ and the output $y_t$:$$h_t = f(W_{hh}h_{t-1} + W_{xh}x_t + b_h)$$The same weights ($W_{hh}$, $W_{xh}$) are reused across all steps in the sequence—this is the principle of weight sharing in time
2. Long Short-Term Memory (LSTM)
LSTMs (Long Short-Term Memory) and GRUs (Gated Recurrent Units) were invented to solve the vanishing gradient problem.
* Forget Gate: Decides what information to throw away from the past state.
* Input Gate: Decides what new information to add to the state.
* Output Gate: Decides what to output based on the updated state.

This gated structure allows gradients to flow smoothly, enabling the network to learn dependencies that span hundreds of time steps.

## 📝 Exercise 1: Implementing an LSTM for Sequence Classification
Your task is to define and run a forward pass through a basic LSTM network for a sequence classification task (e.g., sentiment analysis).

Instructions:

1. Define a network called SentimentLSTM that inherits from `nn.Module`.

2. The network must contain an `nn.LSTM` layer with the following settings:
* input_size: 50 (representing the embedding size of each word/token).
* Hidden_size: 100.
* num_layers: 2.

3. The network should have a final nn.Linear layer to map the final hidden state to 3 classes (e.g., positive, negative, neutral).

4. Define the forward method:
* The input tensor x will have the shape: (`Batch Size, Sequence Length, Input Size`).
* The LSTM output is a tuple: (`output, (h_n, c_n)`). We only care about the final hidden state, $\mathbf{h_n}$.
* Use the final hidden state of the last layer (h_n[-1]) to feed the Linear layer.
5. Run a forward pass with a dummy input: (`Batch=32, Sequence Length=15, Input Size=50`).

6. Print the output logits shape.

In [ ]:
import torch
import  torch.nn as nn

class SentimentLSTM (nn.Module):
  def __init__(self):
    super(SentimentLSTM, self).__init__()
    # LSTM layer with input_size=50, hidden_size=100, num_layers=2
    self.lstm = nn.LSTM(input_size=50, hidden_size=100, num_layers=2, batch_first=True)
    # Linear layer to map the final hidden state to 3 classes
    self.linear = nn.Linear(in_features=100, out_features=3)

  def forward(self, x):
    # x shape: (Batch Size, Sequence Length, Input Size)
    # lstm_output contains all hidden states, (h_n, c_n) contains final hidden and cell states
    lstm_output, (h_n, c_n) = self.lstm(x)

    # Use the final hidden state of the last layer (h_n[-1])
    # h_n has shape (num_layers * num_directions, batch, hidden_size)
    # h_n[-1] gets the hidden state from the last layer for all batches
    logits = self.linear(h_n[-1])
    return logits

  # Instantiate the SentimentLSTM network
  model = SentimentLSTM()

  # Define a dummy input tensor: (Batch=32, Sequence Length=15, Input Size=50)
  dummy_input = torch.randn(32, 15, 50)

  # Perform a forward pass with the dummy input
  output_logits = model(dummy_input)

  # Print the output logits shape
  print("Output logits shape:", output_logits.shape)

Output logits shape: torch.Size([32, 3])


## Module 3: Attention and Transformers
Attention allows the model to selectively focus on the most relevant parts of the input sequence when producing an output, instead of relying on a single final hidden state.
The process is:

1. Calculate Attention Scores by taking the dot product of the Query vector with all Key vectors.

2. Apply Softmax to these scores to create weights (probabilities) that sum to 1.

3. The final output is a weighted sum of the Value vectors, where the weights determine how much the model "pays attention" to each part of the input.



While building a full Transformer from scratch is complex, we can implement the core Scaled Dot-Product Attention mechanism itself.The scaled dot-product attention calculation is defined as:$$\text{Attention}(\mathbf{Q}, \mathbf{K}, \mathbf{V}) = \text{Softmax}(\frac{\mathbf{QK}^T}{\sqrt{d_k}})\mathbf{V}$$where $\sqrt{d_k}$ is the scaling factor to prevent large values from pushing the Softmax into extreme regions.

## 📝 Exercise 2: Implementing Scaled Dot-Product Attention
Your task is to implement the core Scaled Dot-Product Attention function in PyTorch.

Instructions:

1. Write a function scaled_dot_product_attention that accepts three tensors: Q, K, V.
1. Assume the tensors are shaped (Batch, Head, Sequence Length, Feature Dimension).
1. Calculate the raw attention scores using matrix multiplication of $\mathbf{Q}$ and the transpose of $\mathbf{K}$ (over the last two dimensions).
1. Scale the scores by $\frac{1}{\sqrt{d_k}}$, where $d_k$ is the last dimension of the Key tensor ($\mathbf{K}$).
1. Apply Softmax to the scaled scores over the last dimension (the sequence length dimension).
1. Calculate the final attention output by performing matrix multiplication of the attention weights and the Value tensor ($\mathbf{V}$).
1. Run a forward pass using the provided dummy tensors and print the final output shape.

In [4]:
import torch
import torch.nn.functional as F
def scaled_dot_product_attention(Q,V,K):
  k_trans = K.transpose(-2,-1)
  attn_scores = torch.matmul(Q, k_trans)

  d_k = K.shape[-1]
  scaled_attn_scores = attn_scores / torch.sqrt(torch.tensor(d_k, dtype=torch.float32))

  attention_weights = torch.nn.functional.softmax(scaled_attn_scores, dim=-1)

  output = torch.matmul(attention_weights, V)

  return output

# --- Setup Dummy Tensors ---
# B=4 (Batch Size), H=8 (Attention Heads), S=10 (Sequence Length), Dk=64 (Feature Dimension)
batch_size = 4
num_heads = 8
seq_len = 10
d_k = 64 # Feature dimension

# Q, K, V all have the same shape in self-attention: (B, H, S, Dk)
Q = torch.randn(batch_size, num_heads, seq_len, d_k)
K = torch.randn(batch_size, num_heads, seq_len, d_k)
V = torch.randn(batch_size, num_heads, seq_len, d_k)

# Perform a forward pass
output_tensor = scaled_dot_product_attention(Q, K, V)

# Print the output shape
print("Output attention tensor shape:", output_tensor.shape)

Output attention tensor shape: torch.Size([4, 8, 10, 64])


This is the core computational component of the Transformer!

## Module 4: Advanced Model Visualization (Explainability)
Model Explainability (or XAI) techniques help us visualize what the model is looking at. One of the most popular methods for CNNs is Grad-CAM (Gradient-weighted Class Activation Mapping).

1. The Goal of Grad-CAM
Grad-CAM produces a coarse, low-resolution heatmap that highlights the regions in the input image that were most important for the model's classification decision.

2. The Grad-CAM Procedure
Grad-CAM requires two key pieces of information from the network:

1. Activations: The feature maps ($A$) from the final convolutional layer. These are the model's final, rich spatial features.Gradients:
2. The gradients of the classification loss ($L$) with respect to those final feature maps ($\frac{\partial L}{\partial A}$).

## 📝 Exercise 4: Setting up PyTorch Hooks
Before we build the full Grad-CAM function, you need to practice capturing the output of an intermediate layer using a forward hook. This is a fundamental technique in PyTorch inspection.

Instructions:

1. Instantiate a simple nn.Sequential model (like a mini-CNN).

2. Define a dictionary activation_map to store the intermediate tensor.

3. Define a function get_activation that takes the module, input, and output, and saves the output tensor to the activation_map dictionary.

4. Register a Forward Hook on the first ReLU layer of the model, linking it to the get_activation function.

5. Run a forward pass on a dummy input.

6. Print the shape of the tensor saved in the activation_map.

In [5]:
import torch
import torch.nn as nn

# 1. Define a simple neural network model
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.ReLU(),  # This is the layer we will hook
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifier = nn.Linear(32 * 8 * 8, 10) # Assuming input 3x32x32 -> 32x8x8 features

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1) # Flatten for linear layer
        x = self.classifier(x)
        return x

model = SimpleCNN()

# 2. Initialize an empty dictionary to store activations
activation_map = {}

# 3. Define a hook function `get_activation(module, input, output)`
def get_activation(name):
    def hook(model, input, output):
        activation_map[name] = output.detach() # .detach() to prevent memory leaks
    return hook

# 4. Register the forward hook on the first ReLU layer
# Access the first ReLU layer within the 'features' sequential module
# model.features[1] corresponds to the first nn.ReLU()
hook_handle = model.features[1].register_forward_hook(get_activation('first_relu'))

# 5. Create a dummy input tensor
# (Batch_size, Channels, Height, Width) -> (1, 3, 32, 32) for CIFAR-10 like input
dummy_input = torch.randn(1, 3, 32, 32)

# 6. Perform a forward pass through the model
_ = model(dummy_input)

# 7. Print the shape of the stored activation
print("Shape of the captured activation from 'first_relu' layer:", activation_map['first_relu'].shape)

# (Optional) Remove the hook after use to clean up resources
hook_handle.remove()

Shape of the captured activation from 'first_relu' layer: torch.Size([1, 16, 32, 32])


## 📝 Exercise 5: Building and Running Grad-CAM
Your task is to write the complete, runnable code to calculate and print the shape of the final Grad-CAM heatmap. We will simulate the forward/backward pass on your SimpleCNN model.

Instructions:
1. Define a dictionary gradient_map to store the gradients.
2. Define a hook function get_gradient to store the gradient. The backward hook must be registered on the output tensor itself.
3. Run a forward pass through your SimpleCNN and store the output $\mathbf{A}$ of the target layer (model.features[4], the second Conv2D's output ReLU).
4. Register the backward hook on this output tensor $\mathbf{A}$ before the backward pass.
5. Simulate Loss: Since we don't have true labels, select an arbitrary class (e.g., class 5) and backpropagate the gradient of that specific logit: model_output[0][5].backward(retain_graph=True).
6. Calculate the Heatmap: Implement steps 2, 3, and 4 of the Grad-CAM algorithm using the captured $\mathbf{A}$ (from the forward hook) and $\mathbf{G}$ (from the backward hook).
7. Print the final heatmap shape.

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Re-instantiate the model for a clean run if needed, or use the existing one
# (assuming model from Exercise 4 is still in scope)
model = SimpleCNN()
model.eval() # Set model to evaluation mode

# 1. Define a dictionary to store gradients
gradient_map = {}

# 2. Define a hook function to store the gradient (for module backward hook)
def get_gradient(name):
    # For module.register_backward_hook, the hook receives (module, grad_input, grad_output)
    def hook(module, grad_input, grad_output):
        # grad_output is a tuple of gradients for each output; we need the first one.
        gradient_map[name] = grad_output[0].detach()
    return hook

# Define a dictionary to store activations
activation_map_grad_cam = {}
def get_activation_grad_cam(name):
    def hook(model, input, output):
        activation_map_grad_cam[name] = output.detach()
    return hook

# Register forward hook on the target layer (model.features[4], the second ReLU)
forward_hook_handle = model.features[4].register_forward_hook(get_activation_grad_cam('target_layer_activation'))

# Register backward hook on the target layer module (model.features[4])
# This hook will capture gradients flowing back through this module's output.
backward_hook_handle = model.features[4].register_backward_hook(get_gradient('target_layer_gradient'))

# Create a dummy input tensor
dummy_input = torch.randn(1, 3, 32, 32)

# 3. Run a forward pass through your SimpleCNN
model_output = model(dummy_input)

# Get the activation from the target layer (captured by forward hook)
activations = activation_map_grad_cam['target_layer_activation']

# 5. Simulate Loss: Select an arbitrary class (e.g., class 5) and backpropagate
target_class = 5
# The .backward() call will now trigger the module's backward hook.
model_output[:, target_class].sum().backward(retain_graph=True)

# Remove hooks after use to clean up resources
forward_hook_handle.remove()
backward_hook_handle.remove() # Corrected typo here

# Retrieve the captured gradients (G) and activations (A)
gradients = gradient_map['target_layer_gradient']
activations = activation_map_grad_cam['target_layer_activation']

# 6. Calculate the Heatmap (Grad-CAM algorithm)

# Global average pooling of gradients to get neuron importance weights (alpha_k)
# Gradients shape: (1, 32, 8, 8) -> (Batch, Channels, Height, Width)
# Pool over spatial dimensions (Height, Width)
weights = F.adaptive_avg_pool2d(gradients, 1)

# Multiply weights with activations and sum along the channel dimension
# Weights shape: (1, 32, 1, 1)
# Activations shape: (1, 32, 8, 8)
# Resulting heatmap (before ReLU) shape: (1, 32, 8, 8) -> (1, 8, 8) after sum
heatmap = (weights * activations).sum(dim=1, keepdim=True)

# Apply ReLU to the heatmap
heatmap = F.relu(heatmap)

# 7. Print the final heatmap shape
print("Final Grad-CAM heatmap shape:", heatmap.shape)


Final Grad-CAM heatmap shape: torch.Size([1, 1, 16, 16])


That brings us to the end of this session, there are more project end to end pipelines coming up soon